In [2]:
import grpc
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [3]:
!ls /tf_servingd/tf_servingd.sock

/tf_servingd/tf_servingd.sock


In [4]:
channel = grpc.insecure_channel('unix:///tf_servingd/tf_servingd.sock')
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [8]:
x = tf.random.uniform((10, 224, 224, 3))

In [15]:
request = predict_pb2.PredictRequest()
request.model_spec.name = 'mobilenetv2_test1'
request.model_spec.signature_name = 'serving_default'
request.inputs["imgs"].CopyFrom(tf.make_tensor_proto(x))
response = stub.Predict(request, 100)

In [16]:
tf.make_ndarray(response.outputs['output_0'])

array([[7.56978989e-06, 1.21593475e-05, 9.08374786e-05, ...,
        4.82797623e-06, 2.98023224e-06, 1.23620033e-04],
       [1.25765800e-05, 1.66893005e-05, 1.28030777e-04, ...,
        1.15633011e-05, 3.21865082e-06, 1.40070915e-04],
       [8.76188278e-06, 2.37822533e-05, 1.27673149e-04, ...,
        6.25848770e-06, 3.99351120e-06, 2.81095505e-04],
       ...,
       [1.16825104e-05, 1.45435333e-05, 9.62615013e-05, ...,
        6.31809235e-06, 3.93390656e-06, 1.78098679e-04],
       [1.41859055e-05, 1.57952309e-05, 9.43541527e-05, ...,
        7.39097595e-06, 2.50339508e-06, 1.60455704e-04],
       [1.15036964e-05, 1.78217888e-05, 9.50694084e-05, ...,
        7.68899918e-06, 4.58955765e-06, 1.85132027e-04]], dtype=float32)

In [10]:
%%timeit
request = predict_pb2.PredictRequest()
request.model_spec.name = 'mobilenetv2_test1'
request.model_spec.signature_name = 'serving_default'
request.inputs["imgs"].CopyFrom(tf.make_tensor_proto(x))
response = stub.Predict(request, 100)

pred = tf.make_ndarray(response.outputs['output_0'])

22.4 ms ± 875 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
img = tf.image.decode_image(tf.io.read_file('sf.jpg'), 3)
img_letter_box = tf.image.resize_with_pad(img, 608, 608)
img_normalized = img_letter_box / 255.0
imgs = img_normalized[tf.newaxis, ...]

In [7]:
request = predict_pb2.PredictRequest()
request.model_spec.name = 'yolov3'
request.model_spec.signature_name = 'serving_default'
request.inputs["imgs"].CopyFrom(tf.make_tensor_proto(imgs))
response = stub.Predict(request, 1000)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node StatefulPartitionedCall/yolov3/functional_1/conv2d_68/Conv2D}}]]
	 [[PartitionedCall/_761]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node StatefulPartitionedCall/yolov3/functional_1/conv2d_68/Conv2D}}]]
	 [[PartitionedCall/PartitionedCall/TRTEngineOp_1_1]]
0 successful operations.
0 derived errors ignored."
	debug_error_string = "{"created":"@1609344904.950617009","description":"Error received from peer unix:/tf_servingd/tf_servingd.sock","file":"src/core/lib/surface/call.cc","file_line":1061,"grpc_message":"2 root error(s) found.\n  (0) Unknown